<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания № 16


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс PaymentMethod в C#, который будет представлять 
различные способы оплаты. На основе этого класса разработать 2-3 производных 
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из 
классов должны быть реализованы новые атрибуты и методы, а также 
переопределены некоторые методы базового класса для демонстрации 
полиморфизма.
##### Требования к базовому классу PaymentMethod:
• Атрибуты: ID способа оплаты (PaymentMethodId), Название способа оплаты 
(MethodName), Минимальная сумма (MinAmount).

 • Методы:

o ProcessPayment(decimal amount): метод для обработки платежа 
указанной суммы.

o CheckMinimumAmount(decimal amount): метод для проверки 
минимальной суммы платежа.

o GetPaymentDetails(): метод для получения деталей способа оплаты.
##### Требования к производным классам:
1. ОнлайнОплата (OnlinePayment): Должен содержать дополнительные 
атрибуты, такие как URL платежной системы (PaymentUrl). 
Метод ProcessPayment() должен быть переопределен для включения URL 
платежной системы в процесс оплаты.
2. БанковскийПеревод (BankTransfer): Должен содержать дополнительные 
атрибуты, такие как Банковские данные (BankData). 
Метод CheckMinimumAmount() должен быть переопределен для проверки 
минимальной суммы платежа с учетом банковских комиссий.
3. Наличные (CashPayment) (если требуется третий класс): Должен содержать 
дополнительные атрибуты, такие как Место выдачи наличных 
(CashPickupPoint). Метод GetPaymentDetails() должен быть переопределен 
для отображения места выдачи наличных.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте простое, сложное и множественное наследование

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [4]:
public interface IPaymentMethod 
{
    void ProcessPayment(decimal amount);
    bool CheckMinimumAmount(decimal amount);
    string GetPaymentDetails();
}


public class PaymentMethod : IPaymentMethod
{
    public int PaymentMethodId { get; set; }
    public string MethodName { get; set; }
    public decimal MinAmount { get; set; }
    public string Currency { get; set; }
    public DateTime PaymentDate { get; set; }

    public PaymentMethod(int paymentMethodId, string methodName, decimal minAmount, string currency, DateTime paymentDate)
    {
        PaymentMethodId = paymentMethodId;
        MethodName = methodName;
        MinAmount = minAmount;
        Currency = currency;
        PaymentDate = paymentDate;
    }

    public virtual void ProcessPayment(decimal amount)
    {
        Console.WriteLine($"Обработка платежа {amount} {Currency} через {MethodName} на дату {PaymentDate.ToShortDateString()}");
    }

    public virtual bool CheckMinimumAmount(decimal amount)
    {
        return amount >= MinAmount;
    }

    public virtual string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount}, Валюта: {Currency}, Дата: {PaymentDate.ToShortDateString()}";
    }

    public virtual bool IsAvailable()
    {
        return DateTime.Now.Date <= PaymentDate.Date;
    }

    public virtual bool CheckRefundEligibility(decimal amount)
    {
        return amount > MinAmount;
    }
}


public class OnlinePayment : PaymentMethod
{
    public string PaymentUrl { get; set; }
    public string ConfirmationCode { get; set; } 

    public OnlinePayment(int paymentMethodId, string methodName, decimal minAmount, string currency, DateTime paymentDate, string paymentUrl)
        : base(paymentMethodId, methodName, minAmount, currency, paymentDate)
    {
        PaymentUrl = paymentUrl;
    }

    public override void ProcessPayment(decimal amount)
    {
        Console.WriteLine($"Онлайн-платеж {amount} {Currency} через {PaymentUrl}, Код подтверждения: {ConfirmationCode}");
    }

    public string GenerateConfirmationCode()
    {
        ConfirmationCode = new Random().Next(1000, 9999).ToString();
        Console.WriteLine($"Сгенерирован код подтверждения: {ConfirmationCode}");
        return ConfirmationCode;
    }

    public bool VerifyConfirmationCode(string inputCode)
    {
        return inputCode == ConfirmationCode;
    }

    public override string GetPaymentDetails()
    {
        return $"{base.GetPaymentDetails()}, URL: {PaymentUrl}";
    }
}


public class BankTransfer : PaymentMethod
{
    public string BankData { get; set; }
    public decimal TransferFee { get; set; }

    public BankTransfer(int paymentMethodId, string methodName, decimal minAmount, string currency, DateTime paymentDate, string bankData, decimal transferFee)
        : base(paymentMethodId, methodName, minAmount, currency, paymentDate)
    {
        BankData = bankData;
        TransferFee = transferFee;
    }

    public override bool CheckMinimumAmount(decimal amount)
    {
        return amount >= (MinAmount + TransferFee); 
    }

    public decimal CalculateTransferFee(decimal amount)
    {
        return amount * TransferFee / 100;
    }

    public override string GetPaymentDetails()
    {
        return $"{base.GetPaymentDetails()}, через: {BankData}, Комиссия: {TransferFee}%";
    }
}


public class CashPayment : PaymentMethod
{
    public string CashPickupPoint { get; set; }
    public decimal Discount { get; set; } 

    public CashPayment(int paymentMethodId, string methodName, decimal minAmount, string currency, DateTime paymentDate, string cashPickupPoint, decimal discount)
        : base(paymentMethodId, methodName, minAmount, currency, paymentDate)
    {
        CashPickupPoint = cashPickupPoint;
        Discount = discount;
    }

    public decimal ApplyDiscount(decimal amount)
    {
        return amount - (amount * Discount / 100);
    }

    public override string GetPaymentDetails()
    {
        return $"{base.GetPaymentDetails()}, Место выдачи наличных: {CashPickupPoint}, Скидка: {Discount}%";
    }
}


public interface IRefundable
{
    bool RefundPayment(decimal amount);
}


public class RefundableOnlinePayment : OnlinePayment, IRefundable
{
    public RefundableOnlinePayment(int paymentMethodId, string methodName, decimal minAmount, string currency, DateTime paymentDate, string paymentUrl)
        : base(paymentMethodId, methodName, minAmount, currency, paymentDate, paymentUrl)
    {
    }

    public bool RefundPayment(decimal amount)
    {
        Console.WriteLine($"Возврат {amount} через {MethodName}");
        return true;
    }
}


OnlinePayment onlinePayment = new OnlinePayment(1, "Онлайн-оплата", 100, "руб.", DateTime.Now, "https://payment.example.com");
BankTransfer bankTransfer = new BankTransfer(2, "Банковский перевод", 200, "руб.", DateTime.Now, "Сбер", 2);
CashPayment cashPayment = new CashPayment(3, "Оплата наличными", 50, "руб.", DateTime.Now, "Магазин", 5);
RefundableOnlinePayment refundablePayment = new RefundableOnlinePayment(4, "Онлайн-оплата с возвратом", 100, "руб.", DateTime.Now, "https://refundpayment.example.com");

decimal amountToPay = 120;
if (onlinePayment.CheckMinimumAmount(amountToPay))
{
    onlinePayment.GenerateConfirmationCode();
    onlinePayment.ProcessPayment(amountToPay);
}
else
{
    Console.WriteLine($"Минимальная сумма для {onlinePayment.MethodName} не достигнута");
}

if (bankTransfer.CheckMinimumAmount(amountToPay))
{
    decimal fee = bankTransfer.CalculateTransferFee(amountToPay);
    Console.WriteLine($"Комиссия за перевод: {fee}");
    bankTransfer.ProcessPayment(amountToPay);
}
else
{
    Console.WriteLine($"Минимальная сумма для {bankTransfer.MethodName} не достигнута");
}

if (cashPayment.CheckMinimumAmount(amountToPay))
{
    decimal discountedAmount = cashPayment.ApplyDiscount(amountToPay);
    Console.WriteLine($"Сумма после скидки: {discountedAmount}");
    cashPayment.ProcessPayment(discountedAmount);
}
else
{
    Console.WriteLine($"Минимальная сумма для {cashPayment.MethodName} не достигнута");
}

decimal refundAmount = 110;
if (refundablePayment.RefundPayment(refundAmount))
{
    Console.WriteLine("Возврат успешно завершен.");
}
else
{
    Console.WriteLine("Возврат не выполнен.");
}

Console.WriteLine(onlinePayment.GetPaymentDetails());
Console.WriteLine(bankTransfer.GetPaymentDetails());
Console.WriteLine(cashPayment.GetPaymentDetails());
Console.WriteLine(refundablePayment.GetPaymentDetails());


Сгенерирован код подтверждения: 3606
Онлайн-платеж 120 руб. через https://payment.example.com, Код подтверждения: 3606
Минимальная сумма для Банковский перевод не достигнута
Сумма после скидки: 114
Обработка платежа 114 руб. через Оплата наличными на дату 10/7/2024
Возврат 110 через Онлайн-оплата с возвратом
Возврат успешно завершен.
Способ оплаты: Онлайн-оплата, Минимальная сумма: 100, Валюта: руб., Дата: 10/7/2024, URL: https://payment.example.com
Способ оплаты: Банковский перевод, Минимальная сумма: 200, Валюта: руб., Дата: 10/7/2024, через: Сбер, Комиссия: 2%
Способ оплаты: Оплата наличными, Минимальная сумма: 50, Валюта: руб., Дата: 10/7/2024, Место выдачи наличных: Магазин, Скидка: 5%
Способ оплаты: Онлайн-оплата с возвратом, Минимальная сумма: 100, Валюта: руб., Дата: 10/7/2024, URL: https://refundpayment.example.com
